# Imports

In [1]:
import requests
import pandas as pd

# Set the API endpoint URL
api_key = "XXXXXXXXXXX"

# Single Video ID via Search Query
- I've tested these searches in Youtube search bar and they are reliable for this use case (pull back the 1st video that shows up)

In [ ]:
# # Replace {SEARCH_QUERY} with the search term you want to use
# search_query = "Pinkbike Academy Season 3 Episode 10"

# # Make a GET request to the API
# response = requests.get(url)

# # Convert the response to a JSON object
# data = response.json()

# # Extract the video ID from the response
# video_id = data["items"][0]["id"]["videoId"]

# # Print the video ID
# print(f"Video ID: {video_id}")

# Get Multiple Video IDs



In [ ]:
# # Initialize an empty dataframe with two columns: 'search_query' and 'video_id'
df_video_ids = pd.DataFrame(columns=['search_query', 'video_id'])

In [ ]:
search_queries = ['Pinkbike Academy Season 1 Episode 1',
'Pinkbike Academy Season 1 Episode 2',
'Pinkbike Academy Season 1 Episode 3',
'Pinkbike Academy Season 1 Episode 4',
'Pinkbike Academy Season 1 Episode 5',
'Pinkbike Academy Season 1 Episode 6',
'Pinkbike Academy Season 1 Episode 7',
'Pinkbike Academy Season 1 Episode 8',
'Pinkbike Academy Season 1 Episode 9',
'Pinkbike Academy Season 1 Episode 10',
'Pinkbike Academy Season 2 Episode 1',
'Pinkbike Academy Season 2 Episode 2',
'Pinkbike Academy Season 2 Episode 3',
'Pinkbike Academy Season 2 Episode 4',
'Pinkbike Academy Season 2 Episode 5',
'Pinkbike Academy Season 2 Episode 6',
'Pinkbike Academy Season 2 Episode 7',
'Pinkbike Academy Season 2 Episode 8',
'Pinkbike Academy Season 2 Episode 9',
'Pinkbike Academy Season 2 Episode 10',
'Pinkbike Academy Season 3 Episode 1',
'Pinkbike Academy Season 3 Episode 2',
'Pinkbike Academy Season 3 Episode 3',
'Pinkbike Academy Season 3 Episode 4',
'Pinkbike Academy Season 3 Episode 5',
'Pinkbike Academy Season 3 Episode 6',
'Pinkbike Academy Season 3 Episode 7',
'Pinkbike Academy Season 3 Episode 8',
'Pinkbike Academy Season 3 Episode 9',
'Pinkbike Academy Season 3 Episode 10']

In [ ]:
# Set the API endpoint URL
url = f"https://www.googleapis.com/youtube/v3/search?key={api_key}&part=id&type=video"

# Iterate over the list of search query options
for search_query in search_queries:
  # Check if the dataframe already has information for the current search query
  if search_query in df_video_ids['search_query'].values:
    print(f"Information for search query '{search_query}' already exists in the dataframe.")
    continue

  # Set the search query in the URL
  search_url = f"{url}&q={search_query}"

  # Make a GET request to the API
  response = requests.get(search_url)

  # Convert the response to a JSON object
  data = response.json()

  # Extract the video ID from the response
  video_id = data["items"][0]["id"]["videoId"]

  # Add a new row to the dataframe with the search query and video ID
  df_video_ids = df_video_ids.append({'search_query': search_query, 'video_id': video_id}, ignore_index=True)

# Save the updated dataframe to the CSV file
df_video_ids.to_csv('df_video_ids.csv', index=False)

df_video_ids.head(3)

,search_query,video_id
0,Pinkbike Academy Season 1 Episode 1,NHGt09eIyoI
1,Pinkbike Academy Season 1 Episode 2,ECM28aYL_yA
2,Pinkbike Academy Season 1 Episode 3,oy7vXU85amw


In [ ]:
df_video_ids['video_id'].nunique()

30

In [ ]:
video_ids = df_video_ids['video_id'].unique().tolist()

# Get Comments from Multiple Video IDs

In [ ]:
# Set the API endpoint URL
url = f"https://www.googleapis.com/youtube/v3/commentThreads?key={api_key}&part=snippet"

# Set the maximum number of comments to retrieve (up to a maximum of 100)
max_results = 100

# Set the initial value for the next_page_token variable
next_page_token = ""

# Create a list of video IDs
video_ids = video_ids

# Initialize an empty dictionary to store the comments for each video
comments = {}

# Iterate over the list of video IDs
for video_id in video_ids:
  # Initialize an empty list to store the comments for the current video
  video_comments = []

  # Set the URL to retrieve comments for the current video
  video_url = f"{url}&videoId={video_id}"

  while True:
    # Make a GET request to the API
    response = requests.get(video_url)

    # Convert the response to a JSON object
    data = response.json()

    # Extract the comments from the response
    video_comments += data["items"]

    # Check if there is a next page of comments
    if "nextPageToken" in data:
      # Set the value of the next_page_token variable
      next_page_token = data["nextPageToken"]
      # Update the URL to retrieve the next page of comments
      video_url = f"{video_url}&pageToken={next_page_token}"
    else:
      # If there are no more pages of comments, break out of the loop
      break

  # Add the list of comments for the current video to the dictionary
  comments[video_id] = video_comments

# Print the total number of comments for each video
for video_id, video_comments in comments.items():
  print(f"Total comments for video {video_id}: {len(video_comments)}")

In [ ]:
# Initialize an empty list to store the comment data
comment_data = []

# Iterate over the comments for each video
for video_id, video_comments in comments.items():
  # Iterate over the comments for the current video
  for comment in video_comments:
    # Add the comment data to the list
    comment_data.append({
        "video_id": video_id,
        "author_name": comment["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"],
        "author_channel_id": comment["snippet"]["topLevelComment"]["snippet"]["authorChannelId"]["value"],
        "text": comment["snippet"]["topLevelComment"]["snippet"]["textOriginal"],
        "like_count": comment["snippet"]["topLevelComment"]["snippet"]["likeCount"]
    })

# Create a pandas dataframe from the comment data
df = pd.DataFrame(comment_data)

# Print the first few rows of the dataframe
print(df.head())


      video_id            author_name         author_channel_id  \
0  NHGt09eIyoI               Dru Down  UCcBo8kc_YcthJXcROge-GSw   
1  NHGt09eIyoI           Synergy Tech  UCGwoFN4nnZ3gaLLMqtyz5oQ   
2  NHGt09eIyoI        Expert Noob MTB  UC7EV_AJr8KVw7_sKL9v1WRw   
3  NHGt09eIyoI  Knight Rider Explorer  UCdP8uksWxG82gmxTL8tn9Ag   
4  NHGt09eIyoI               henry06x  UCOVxUNak6NNPihdDssLbAfw   

                                                text  like_count  
0  Ben looks like a tool. Let's see the rest season.           0  
1                                         Great show           0  
2  Angie cant accept the fact that she lost and N...           0  
3                                 Nicole i LOVED YOU           0  
4  I woulda been excited seeing the bike needing ...           0  


In [ ]:
df['video_id'].nunique()

30

In [ ]:
df_sentiment = df.merge(df_video_ids, on = 'video_id', how = 'inner')
df_sentiment.sort_values('like_count', ascending = False).head(5)

,video_id,author_name,author_channel_id,text,like_count,search_query
4130,TlqkncjO1x0,El Danno,UCcxVoeyq1eO-t5-o4WcOijw,The answer “quit my job to be here” No one els...,1040,Pinkbike Academy Season 1 Episode 10
7443,TBsbPI-emDI,Sergei Kulichkov,UCfCHE5LX-cze-VvLmyxE5Hw,Based on the instagram pages Brad is now racin...,1024,Pinkbike Academy Season 2 Episode 10
2943,4-qUnyl5Z14,kalekalekale,UCMwoOkeXWlu8cqKg7p4KRhQ,Addison wouldve been perfect with this challen...,910,Pinkbike Academy Season 1 Episode 7
632,NHGt09eIyoI,Albert Smyser,UCrCGSvkX_D4A-8K8XaOqU6A,I hate reality TV......when does the next epis...,848,Pinkbike Academy Season 1 Episode 1
6362,VBlrgylVKTI,Downhill Dummy,UC8_oMiMU5gWLsyK1o4ws6iA,"Cam hurts himself badly, flats for a loss, and...",791,Pinkbike Academy Season 2 Episode 6


In [ ]:
df_sentiment.to_csv("all_comments_out.csv")

## Word Counter, Total

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from collections import Counter
from nltk.corpus import stopwords

# Load the data into a Pandas DataFrame
df = df_sentiment

# Select the column that you want to process
column = df['text']

# Split the text in the column into a list of words
words = column.apply(lambda x: x.split())

# Flatten the list of lists into a single list
flat_list = [item for sublist in words for item in sublist]

# Remove general words and stopwords from the list
filtered_words = [word for word in flat_list if word not in stopwords.words('english') and word.isalpha()]

# Count the occurrences of each word
word_counts = Counter(filtered_words)

# Sort the word counts in descending order
sorted_word_counts = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)

# Print the top 10 most common words
print(sorted_word_counts[:10])

[('I', 2610), ('like', 952), ('get', 520), ('see', 517), ('show', 515), ('This', 508), ('would', 500), ('Cody', 460), ('one', 457), ('season', 455)]


In [ ]:
sorted_word_counts[:10]

[('I', 2610),
 ('like', 952),
 ('get', 520),
 ('see', 517),
 ('show', 515),
 ('This', 508),
 ('would', 500),
 ('Cody', 460),
 ('one', 457),
 ('season', 455)]

In [ ]:
kw_10_mentions = {k: v for k, v in word_counts.items() if v >= 10}

In [ ]:
len(kw_10_mentions)

1221

# Word Counter, Grouped By Episode

In [ ]:
df = df_sentiment

# Define the list of keywords that you want to search for
kw_10_mentions = [word.lower() for word in kw_10_mentions]
kw_10_mentions = [word for word in kw_10_mentions if len(word) > 2]

# Define the character to add
character = ' '

# Use a list comprehension and the concatenation operator to add the character to each value
kw_10_mentions = [character + str(value) + character for value in kw_10_mentions]

keywords = kw_10_mentions

# Group the data by the search_query column
grouped_df = df.groupby('search_query')

# Initialize an empty list to store the results
results = []

# Loop through each group
for search_query, group in grouped_df:
  # Loop through each keyword
  for keyword in keywords:
    # Count the number of rows that contain the keyword in the text column
    count = group['text'].str.contains(keyword, case = False).sum()
    # Append the search query, keyword, and count to the results list
    results.append({'search_query': search_query, 'keyword': keyword, 'count': count})

# Convert the results list into a Pandas dataframe
result_df = pd.DataFrame(results)

In [ ]:
result_df.sort_values('count', ascending = False).head(8)

,search_query,keyword,count
7202,Pinkbike Academy Season 1 Episode 6,the,264
7412,Pinkbike Academy Season 1 Episode 6,the,264
1397,Pinkbike Academy Season 1 Episode 10,the,262
1607,Pinkbike Academy Season 1 Episode 10,the,262
24617,Pinkbike Academy Season 3 Episode 10,the,257
24827,Pinkbike Academy Season 3 Episode 10,the,257
446,Pinkbike Academy Season 1 Episode 1,the,203
236,Pinkbike Academy Season 1 Episode 1,the,203


# Output Data
- Methodology note: this is currently looking for total mentions of keywords. So, for example, if Tarmo is mentioned 5X in 1 comment, it will count Tarmo 5 times

In [ ]:
len(result_df)

34830

# Extract Season and Episode
- Function to be applied later in code

In [ ]:
import re

def extract_season_episode(text):
    # Use a regular expression to extract the season and episode numbers
    match = re.search(r'Season (\d+) Episode (\d+)', text)
    
    if match:
        # If a match was found, return the season and episode numbers as a tuple
        return int(match.group(1)), int(match.group(2))
    else:
        # If no match was found, return None
        return None

# Test the function
print(extract_season_episode('Pinkbike Academy Season 1 Episode 10'))  # Output: (1, 10)
print(extract_season_episode('Pinkbike Academy Season 2 Episode 1'))  # Output: (2, 1)
print(extract_season_episode('Pinkbike Academy'))  # Output: None

(1, 10)
(2, 1)
None


In [ ]:
# Create a new column 'season_episode' with the season and episode numbers
result_df['season_episode'] = result_df['search_query'].apply(extract_season_episode)

# Create a new column 'season' with the season numbers
result_df['season'] = result_df['season_episode'].apply(lambda x: x[0] if x else None)

# Create a new column 'episode' with the episode numbers
result_df['episode'] = result_df['season_episode'].apply(lambda x: x[1] if x else None)
result_df.head(1)

,search_query,keyword,count,season_episode,season,episode
0,Pinkbike Academy Season 1 Episode 1,ben,32,"(1, 1)",1,1


In [ ]:
count_comments = df_sentiment['search_query'].value_counts().reset_index()
count_comments['episode_total_comments'] = count_comments['search_query']
del count_comments['search_query']
count_comments['search_query'] = count_comments['index']
del count_comments['index']

count_comments.head(1)

,episode_total_comments,search_query
0,698,Pinkbike Academy Season 1 Episode 10


In [ ]:
len(count_comments)

30

In [ ]:
result_df_out = result_df.merge(count_comments, on = 'search_query', how = 'inner')

In [ ]:
result_df_out.to_csv('keyword_count_per_episode.csv')

In [ ]:
# result_df_out[result_df_out['keyword'] == 'brody']

In [ ]:
result_df_out.head(1)

,search_query,keyword,count,season_episode,season,episode,episode_total_comments
0,Pinkbike Academy Season 1 Episode 1,ben,32,"(1, 1)",1,1,657
